In [131]:
import tensorflow.keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [132]:
!ls -la

total 56
drwxr-xr-x   5 guillaumevanderwinden  staff    160 Nov 25 12:28 .
drwxr-xr-x  19 guillaumevanderwinden  staff    608 Nov 24 18:03 ..
drwxr-xr-x   3 guillaumevanderwinden  staff     96 Nov 24 14:53 .ipynb_checkpoints
-rw-r--r--   1 guillaumevanderwinden  staff      0 Nov 24 12:27 .keep
-rw-r--r--   1 guillaumevanderwinden  staff  26069 Nov 25 12:28 gui-preproc.ipynb


In [133]:
!pwd

/Users/Gui/code/GuiVdwinden/greeneye/notebooks


# Loading X (images) and y (labels)

In [149]:
data_1000 = []

for i in range(1000):
    image = load_img(f'../Sample_train700/train_{i}.jpg', color_mode='rgb')
    array = img_to_array(image, data_format='channels_last')
    data_1000.append(array)

In [151]:
X_sample_train = np.array(data_1000[:700])
X_sample_train.shape

(700, 256, 256, 3)

In [153]:
encoded_target = pd.read_csv('../EncodedData.csv')

In [154]:
encoded_target = pd.DataFrame(data=encoded_target)

In [152]:
alltags = [ 'clear', 'cloudy', 'haze', 'partly cloudy',            
           'agriculture', 'artisinal mine', 'bare ground', 'blooming', 
           'blow down', 'cultivation', 'habitation', 'primary', 'road', 
        'selective logging', 'conventional mine', 'slashu burn','water']

In [155]:
encoded_target.head()

,image_name,tags,taglist,clear,cloudy,haze,partly cloudy,agriculture,artisinal mine,bare ground,...,blow down,cultivation,habitation,primary,road,selective logging,conventional mine,slashu burn,water,result_id
0,train_0,haze primary,"['haze', 'primary']",0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,'00100000000100000'
1,train_1,agriculture clear primary water,"['clear', 'agriculture', 'primary', 'water']",1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,'10001000000100001'
2,train_2,clear primary,"['clear', 'primary']",1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,'10000000000100000'
3,train_3,clear primary,"['clear', 'primary']",1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,'10000000000100000'
4,train_4,agriculture clear habitation primary road,"['clear', 'agriculture', 'habitation', 'primar...",1,0,0,0,1,0,0,...,0,0,1,1,1,0,0,0,0,'10001000001110000'


In [156]:
target_1000 = encoded_target[alltags].to_numpy()
y_sample_train = target_1000[:700]

In [157]:
X_sample_train = X_sample_train / 255.

In [158]:
X_sample_train.shape

(700, 256, 256, 3)

# Building a first baseline model

In [159]:
from tensorflow.keras import models
from tensorflow.keras import layers 


def initialize_model():
    model = models.Sequential()
    model.add(layers.Conv2D(16, (3, 3), padding='same', input_shape=(256, 256, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(3, 3)))

    model.add(layers.Flatten())
    model.add(layers.Dense(120, activation='relu'))
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(17, activation='softmax'))
    
    return model

In [160]:
model = initialize_model()

model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 128, 128, 64)      9280      
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 64, 64, 128)       73856     
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 21, 21, 128)       0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 56448)           

In [161]:
from tensorflow.keras import optimizers

def compile_model(model):
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [162]:
from tensorflow.keras.callbacks import EarlyStopping

model = initialize_model()
model = compile_model(model)

es = EarlyStopping(patience=5, verbose=1)

history = model.fit(X_sample_train, y_sample_train, 
                    validation_split=0.3,
                    callbacks=[es], 
                    epochs=50, 
                    batch_size=32)

Epoch 1/50
16/16 [==============================] - 120s 8s/step - loss: 55.2897 - accuracy: 0.1902 - val_loss: 296.4903 - val_accuracy: 0.1090
Epoch 2/50
16/16 [==============================] - 121s 8s/step - loss: 4406.7666 - accuracy: 0.3088 - val_loss: 12816.4785 - val_accuracy: 0.1090
Epoch 3/50
16/16 [==============================] - 110s 7s/step - loss: 96359.7891 - accuracy: 0.2781 - val_loss: 229194.3125 - val_accuracy: 0.1090
Epoch 4/50
16/16 [==============================] - 111s 7s/step - loss: 1000827.5625 - accuracy: 0.2904 - val_loss: 2197946.0000 - val_accuracy: 0.1090
Epoch 5/50
16/16 [==============================] - 82s 5s/step - loss: 7113829.5000 - accuracy: 0.2434 - val_loss: 13492280.0000 - val_accuracy: 0.1090
Epoch 6/50
16/16 [==============================] - 68s 4s/step - loss: 30993142.0000 - accuracy: 0.2740 - val_loss: 58716840.0000 - val_accuracy: 0.1090
Epoch 00006: early stopping


In [183]:
X_sample_test = np.array(data_1000[700:])
y_sample_test = target_1000[700:1000]

In [168]:
y_sample_test.shape

(300, 17)

In [180]:
len(X_sample_test)

300

# Evaluating

In [184]:
res = model.evaluate(X_sample_test, y_sample_test, verbose=0)

In [185]:
res

[13696888832.0, 0.12999999523162842]

In [190]:
y_pred = model.predict(X_sample_test)

In [191]:
y_pred

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [196]:
import tensorflow
def f2_score(y_true, y_pred):
    y_true = tensorflow.cast(y_sample_test, "int32")
    y_pred = tensorflow.cast(tensorflow.round(y_pred), "int32") # implicit 0.5 threshold via tf.round
    y_correct = y_true * y_pred
    sum_true = tensorflow.reduce_sum(y_true, axis=1)
    sum_pred = tensorflow.reduce_sum(y_pred, axis=1)
    sum_correct = tensorflow.reduce_sum(y_correct, axis=1)
    precision = sum_correct / sum_pred
    recall = sum_correct / sum_true
    f_score = 5 * precision * recall / (4 * precision + recall)
    f_score = tensorflow.where(tensorflow.math.is_nan(f_score), tensorflow.zeros_like(f_score), f_score)
    return tensorflow.reduce_mean(f_score)

In [197]:
f2 = f2_score(y_sample_test, y_pred)
f2

<tf.Tensor: shape=(), dtype=float64, numpy=0.462325696090402>

# Second compiling with F2 score

In [204]:
from tensorflow.keras import optimizers

def compile_model(model):
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=[f2_score])
    return model

In [203]:
model = initialize_model()
model2 = compile_model(model)

history = model2.fit(X_sample_train, y_sample_train, 
                    validation_split=0.3, 
                    epochs=30, 
                    batch_size=32)

Epoch 1/30


ValueError: in user code:

    /Users/Gui/.pyenv/versions/3.7.7/envs/green-eye/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Users/Gui/.pyenv/versions/3.7.7/envs/green-eye/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/Gui/.pyenv/versions/3.7.7/envs/green-eye/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/Gui/.pyenv/versions/3.7.7/envs/green-eye/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/Gui/.pyenv/versions/3.7.7/envs/green-eye/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _